In [ ]:
'''Import necessary libraries'''

import numpy as np
import math
rng = np.random.default_rng()  
import matplotlib.pylab as plt
import random

In [ ]:
'''generate the array of temperatures of interest
logarithmic, so more points at low temperatures'''
n = np.linspace(0,100,101)
Tmax = 5
Tmin = 0.1
b = (Tmax/Tmin)**(1/100)
Tn = Tmax/(b**n) 
print(Tn)

In [ ]:
''' Initialise initial random lattice'''

def init_spin(width):
    '''Produce an initial lattice with random spins 1 or -1'''
    lat = rng.integers(0,2,(width,width))*2-1
    
    return lat

In [ ]:
'''generate a 4-d array of the lattice to map each lattice point to its neighbours
speeds up the computation by having a look-up table'''
def nearestneighbourstable(width):
    nn = np.zeros((width,width,2,7))
    for i in range(0,width):
        for j in range(0,width):
            nn[i,j,:,0] = neighbouring_sites1(i,j,width)[0]
            nn[i,j,:,1] = neighbouring_sites1(i,j,width)[1]
            nn[i,j,:,2] = neighbouring_sites1(i,j,width)[2]
            nn[i,j,:,3] = neighbouring_sites1(i,j,width)[3]
            nn[i,j,:,4] = neighbouring_sites2(i,j,width)[1]
            nn[i,j,:,5] = neighbouring_sites2(i,j,width)[2]
            nn[i,j,:,6] = neighbouring_sites2(i,j,width)[3]

    return nn

In [ ]:
''' Generating the Metropolis-Hastings algorithm'''

def neighbouring_sites1(i,j,width):
    '''Returns the coordinates of the 4 spins around 1 of the lattice sites. Takes into account periodic boundary conditions.''' 
    if (i+j)%2==0:
        return [i,j], [(i + 1) % (width), j], [i, (j+1) % (width)], [(i + 1) % (width), (j+1) % (width)]
    else:
        return [i,j], [(i + 1) % (width), j], [i, (j-1) % (width)], [(i + 1) % (width), (j-1) % (width)]
def neighbouring_sites2(i,j,width):
    '''Returns the coordinates of the 4 spins around the other lattice site. Takes into account periodic boundary conditions.''' 
    if (i+j)%2==0:
        return [i,j], [(i - 1) % (width), j], [i, (j-1) % (width)], [(i - 1) % (width), (j-1) % (width)]
    else:
        return [i,j], [(i - 1) % (width), j], [i, (j+1) % (width)], [(i - 1) % (width), (j+1) % (width)]
    
def neighbouring_spins_sum1old(i,j,lattice,width,nntable):
    '''Sums the total spin around 1 lattice site.'''
    H = 0
    for k in range(0,4):
        x = nntable[i,j,0,k]
        y = nntable[i,j,1,k]
        H += lattice[int(x),int(y)]
    return H

def neighbouring_spins_sum2old(i,j,lattice,width,nntable):
    '''Sums the total spin around the other lattice site.'''
    H = 0
    for k in (0,4,5,6):
        x = nntable[i,j,0,k]
        y = nntable[i,j,1,k]
        H += lattice[int(x),int(y)]
    return H


def neighbouring_spins_sum1new(i,j,lattice,width,nntable):
    '''Sums the total spin around 1 lattice site, if the spin is flipped.'''
    H = 0
    lattice[i,j] = -lattice[i,j]
    for k in range(0,4):
        x = nntable[i,j,0,k]
        y = nntable[i,j,1,k]
        H += lattice[int(x),int(y)]
    lattice[i,j] = -lattice[i,j]
    return H

def neighbouring_spins_sum2new(i,j,lattice,width,nntable):
    '''Sums the total spin around the other lattice site, if the spin is flipped.'''
    H = 0
    lattice[i,j] = -lattice[i,j]
    for k in (0,4,5,6):
        x = nntable[i,j,0,k]
        y = nntable[i,j,1,k]
        H += lattice[int(x),int(y)]
    lattice[i,j] = -lattice[i,j]
    return H

'''need 2 more functions with transverse field, as we will need to know if 
any monopoles are created at the perturbed lattice sites when a 
spin flip is attempted at one of its neighbours: not necessary for classical case'''

def preferred_neighbouring_spins_sum1new(i,j,k,l,lattice,width,nntable):
    H = 0
    lattice[i,j] = -lattice[i,j]
    for a in range(0,4):
        x = nntable[k,l,0,a]
        y = nntable[k,l,1,a]
        H += lattice[int(x),int(y)]
    lattice[i,j] = -lattice[i,j]
    return H
def preferred_neighbouring_spins_sum2new(i,j,k,l,lattice,width,nntable):
    H = 0
    lattice[i,j] = -lattice[i,j]
    for a in (0,4,5,6):
        x = nntable[k,l,0,a]
        y = nntable[k,l,1,a]
        H += lattice[int(x),int(y)]
    lattice[i,j] = -lattice[i,j]
    return H


In [ ]:
def compute_deltaE(i,j,lattice,pref,width,J,h,T,nnatble,nnpref):
    '''Computes the energy difference between the old and new state if spin [i,j] would be flipped.'''
    sn1 = neighbouring_spins_sum1new(i,j,lattice,width,nntable)
    sn2 = neighbouring_spins_sum2new(i,j,lattice,width,nntable)
    so1 = neighbouring_spins_sum1old(i,j,lattice,width,nntable)
    so2 = neighbouring_spins_sum2old(i,j,lattice,width,nntable)
    deltaE = J/2*(sn1*sn1 + sn2*sn2 - so1*so1 - so2*so2)
    
    '''next section not necessary for classical case, it checks if a 1-monopole
    state is created or destroyed at a perturbed state, and changes the energy accordingly'''
    
    if [i,j] in nnpref:
        a = nnpref.index([i,j])
        k = nnpref[a-a%7][0]
        l = nnpref[a-a%7][1]
        psn1 = preferred_neighbouring_spins_sum1new(i,j,k,l,lattice,width,nntable)
        psn2 = preferred_neighbouring_spins_sum2new(i,j,k,l,lattice,width,nntable)
        pso1 = neighbouring_spins_sum1old(k,l,lattice,width,nntable)
        pso2 = neighbouring_spins_sum2old(k,l,lattice,width,nntable)        
        if psn1*psn1 + psn2*psn2 == 0 and pso1*pso1 + pso2*pso2 == 4:
            deltaE += h + 2*J
        if pso1*pso1 + pso2*pso2 == 0 and psn1*psn1 + psn2*psn2 == 4:
            deltaE += -h - 2*J
    return deltaE

def attempt_spin_flip(lattice,pref,width,J,h,T,nntable,nnpref):
    '''Applies the Metropolis-Hastings algorithm to try and flip a spin.'''
    i,j = rng.integers(0,width,2)
    E = compute_deltaE(i,j,lattice,pref,width,J,h,T,nntable,nnpref)
    if E <= 0.1 or rng.random() < np.exp(-E/T):
        lattice[i,j] = -lattice[i,j]

In [ ]:
''' Perform 1 time step on the lattice'''
def monte_step(lattice,pref,width,J,h,T,nntable,nnpref):
    for i in range(0, int(width*width)):
        attempt_spin_flip(lattice,pref,width,J,h,T,nntable,nnpref)

In [ ]:
'''generates look-up table for the neighbours of perturbed sites'''

def pref_neighbours(width,pref):
    nnpref = []
    for i in range(0,width):
        for j in range(0,width):
            if (i,j) in pref:
                nnpref.append(neighbouring_sites1(i,j,width)[0])
                nnpref.append(neighbouring_sites1(i,j,width)[1])
                nnpref.append(neighbouring_sites1(i,j,width)[2])
                nnpref.append(neighbouring_sites1(i,j,width)[3])
                nnpref.append(neighbouring_sites2(i,j,width)[1])
                nnpref.append(neighbouring_sites2(i,j,width)[2])
                nnpref.append(neighbouring_sites2(i,j,width)[3])
    return nnpref

In [ ]:
'''Code to find the physical quantities of an equilibrated lattice'''


#first copy the array of relaxation times
tau_100 = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
           2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 
           3, 3, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 6, 7, 8, 9, 
           10, 12, 13, 15, 17, 20, 24, 27, 30, 39, 47, 55, 
           65, 79, 88, 133, 165, 232, 272, 344, 427, 592, 
           694, 900, 1263, 1787]

#start the simulations for different transverse field strengths

for gamma in [3,6, "inf"]:
    
    #set the physical quantities of the system
    J = 1
    if gamma == "inf":
        h = 0
    else:
        h = -np.sqrt(4*J*J + gamma*gamma) + gamma
    hist = 10
    width = 5
    
    #generate the look-up table and create the random perturbed sites
    nntable = nearestneighbourstable(width)
    prefnumber = int(width*width/25)
    pref = []
    for k in range(0,10000):
        overlap=0
        for i in range (0,prefnumber):
            a = random.randint(0,width-1)
            b = random.randint(0,width-1)
            pref.append((a,b))
        for M in pref:
            i = M[0]
            j = M[1]
            if (i+j)%2==0:
                a = [((i+1)% (width),j), ((i-1)% (width),j), (i,(j+1)% (width)), (i,(j-1)% (width)), ((i+1)% (width),(j+1)% (width)),((i-1)% (width),(j-1)% (width))]
            else:
                a = [((i+1)% (width),j), ((i-1)% (width),j), (i,(j+1)% (width)), (i,(j-1)% (width)), ((i+1)% (width),(j-1)% (width)),((i-1)% (width),(j+1)% (width))]
            if set(a).intersection((set(pref)))!=set([]):
                overlap+=1
            else:
                if len(pref) != len(set(pref)):
                    overlap +=1
        if overlap>0:
            pref = []
        else:
            break
    print(pref)
    nnpref = pref_neighbours(width,pref)
    
    
    #initialise the matrix for the quantities and thermalise the initial lattice
    quant_100 = np.zeros((len(tau_100),7))
    S = init_spin(width)
    for i in range(0,5):
        monte_step(S,pref,width,J,h,Tn[0],nntable,nnpref)
        
        
    #find the quantities for each temperature seperately

    for i in range(0,len(tau_100)):
        E = 0
        M = 0
        Nm = 0
        EE = 0
        MM = 0
        Nmp = 0
        for j in range(0,hist):
            S0 = np.copy(S)
            for k in range(0,2*tau_100[i]):
                monte_step(S0,pref,width,J,h,Tn[i],nntable,nnpref)   #equilibrates the lattice
                
            #go over the lattice sites one by one and calculate the physical quantities of interest
            for l in range(0,width):
                for m in range(0,width):
                    s1 = neighbouring_spins_sum1old(l,m,S0,width,nntable)
                    s2 = neighbouring_spins_sum2old(l,m,S0,width,nntable)
                    M += ((-1)**m)*S0[l,m]         #magnetisation, prefactor because of how the spins are defined in spin ice
                    if (l,m) in pref:
                        if s1*s1 + s2*s2 == 4:
                            E += -h - J/2 *(s1*s1 + s2*s2)      #energy correction for perturbed sites
                    if (l+m)%2==0:        #divide by 2 as there are double the amount of spin sites as potential monopole sites
                        n = (s1)*(s1)
                        E += n*J/2        #energy
                        if n >= 0.5:
                            Nm += 1       #monopole number
                            if (l,m) in pref or ((l+1)%width,m) in pref or ((l+1)%width,(m+1)%width) in pref or (l,(m+1)%width) in pref:
                                Nmp += 1       #monopole number near preferred sites
                                
            EE += E*E
            MM += M*M
            
        #average the quantities over the histories and print the final output for each gamma
        E_n = E/hist
        M_n = M/hist
        Nm_n = Nm/hist
        MM_n = MM/hist
        EE_n = EE/hist
        Nmp_n = Nmp/hist
        quant_100[i,0] = Tn[i]
        quant_100[i,1] = E_n
        quant_100[i,2] = M_n
        quant_100[i,3] = Nm_n
        quant_100[i,4] = Nmp_n
        quant_100[i,5] = EE_n
        quant_100[i,6] = MM_n    
    print(repr(quant_100))
  